In [1]:
import mysql.connector
password1 = input("please enter your password for Mysql")
#cursor.execute('create database Krishna;')

please enter your password for Mysql9494


In [47]:
import tkinter as tk
from tkinter import Entry, Button, StringVar, font as tkFont, Menu
import mysql.connector

# Create a function to fetch project procedure
def fetch_information(option):
    try:
        user_query = user_query_var.get()
        query = ""

        if option == "Procedure":
            query = f"SELECT ProjectProcedure FROM prakalp_nirdesh WHERE Project_Name = %s"
        elif option == "Path":
            query = f"SELECT Paths FROM prakalp_nirdesh WHERE Project_Name = %s"
        elif option == "Head":
            query = f"SELECT Project_Member FROM prakalp_nirdesh WHERE Project_Name = %s"
        elif option == "Start Date":
            query = f"SELECT Start_Date FROM prakalp_nirdesh WHERE Project_Name = %s"
        elif option == "End Date":
            query = f"SELECT End_Date FROM prakalp_nirdesh WHERE Project_Name = %s"

        if query:
            cursor.execute(query, (user_query,))
            result = cursor.fetchone()
            if result is not None:
                result_text.delete(1.0, tk.END)
                result_text.insert(tk.END, f'{option} for {user_query} is: {result[0]}')
            else:
                result_text.delete(1.0, tk.END)
                result_text.insert(tk.END, f'No {option} information found for {user_query}')
        else:
            result_text.delete(1.0, tk.END)
            result_text.insert(tk.END, "No information found.")

    except Exception as e:
        print("An error occurred:", str(e))

krishnadb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password1,
    database="Krishna"
)

cursor = krishnadb.cursor()
cursor.execute('use Krishna;')

root = tk.Tk()
root.title("Project Details")

# Define Tkinter variables to store user input and results
user_query_var = StringVar()

# Create labels for the input fields
font_settings = tkFont.Font(family="Helvetica", size=20, slant="italic", weight="bold")
user_query_label = tk.Label(root, text="Enter Project Name:", font=font_settings, fg="red")

# Create Entry widgets for user input
entry_width = 60
entry_bg_color = "light yellow"
user_query_entry = Entry(root, textvariable=user_query_var, width=entry_width, bg=entry_bg_color)

# Create Text widgets for displaying the results
result_font = tkFont.Font(family="Helvetica", size=14, slant="italic")
result_text = tk.Text(root, wrap=tk.WORD, width=entry_width, height=5, font=result_font, fg="black")

# Create a menu bar
menubar = Menu(root)
root.config(menu=menubar)

# Create a "Fetch" menu with different options
fetch_menu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Menu", menu=fetch_menu)
fetch_menu.add_command(label="Project Procedure", command=lambda: fetch_information("Procedure"))
fetch_menu.add_command(label="Project Path", command=lambda: fetch_information("Path"))
fetch_menu.add_command(label="Project Member", command=lambda: fetch_information("Head"))
fetch_menu.add_command(label="Project Start Date", command=lambda: fetch_information("Start Date"))
fetch_menu.add_command(label="Project End Date", command=lambda: fetch_information("End Date"))

# Create a button to trigger the database query4
get_details_button = Button(root, text="Get Details", command=lambda: fetch_information("Procedure"), font=tkFont.Font(size=18, slant="italic"), fg="green")

# Place widgets on the grid
user_query_label.grid(row=0, column=0)
user_query_entry.grid(row=0, column=1)
get_details_button.grid(row=1, column=0, columnspan=2)
result_text.grid(row=2, column=0, columnspan=2)

root.mainloop()


In [4]:
import tkinter as tk
from tkinter import Entry, Button, Label
import mysql.connector
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# ... (Previous code remains the same
db = mysql.connector.connect(
            host="localhost",
            user="root",
            password=password1,
            database="Krishna"
        )

cursor = db.cursor()

# Preprocess user query
def preprocess_query(user_query):
    words = nltk.word_tokenize(user_query)
    words = [word for word in words if word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return words

def get_project_info(project_name, info_type):
    try:
        if info_type not in ["path", "procedure", "deadline", "head"]:
            return "Invalid info type. Please specify 'path', 'procedure', 'deadline', or 'head'."
    
        if 'path' in info_type:
            column_name = 'Paths'
        elif 'procedure' in info_type:
            column_name = 'ProjectProcedure'
        elif 'head' in info_type:
            column_name = 'Project_Member'
        print(project_name)
        query = f"SELECT {column_name} FROM prakalp_nirdesh WHERE Project_Name like %s"
        print("Query:", query)
        cursor.execute(query, (project_name,))

        info = cursor.fetchall()
        if info:
            return info[0]
        else:
            return f"No {info_type} found for {project_name}."
    except Exception as e:
        print("An error occurred while querying the database:", str(e))
        return "An error occurred while fetching information."

# Function to handle button click
def handle_button_click():
    user_input = user_input_entry.get()
    user_input_entry.delete(0, tk.END)
    user_input = user_input.lower()

    if 'path' in user_input:
        column_name = 'path'
    elif 'procedure' in user_input:
        column_name = 'procedure'
    elif 'head' in user_input:
        column_name = 'head'
    
    # Check if the user input contains the query format "of Project1" or "of Project 1"
    if "of project" in user_input:
        project_name = user_input.split("of project")[1].strip()

        info = get_project_info(project_name, column_name)  # Assuming the user wants the path
        if info:
            add_message(f"You: {user_input}\n", "black", "red")
            add_message(f"Chatbot: The {column_name} for {project_name} is: {info}\n", "black", "white")
        else:
            add_message(f"Chatbot: I couldn't find information for {project_name}.\n", "black", "white")
    elif "for project" in user_input:
        project_name = user_input.split("for project")[1].strip()

        info = get_project_info(project_name, column_name)  # Assuming the user wants the path
        if info:
            add_message(f"You: {user_input}\n", "black", "red")
            add_message(f"Chatbot: The {column_name} for {project_name} is: {info}\n", "black", "white")
        else:
            add_message(f"Chatbot: I couldn't find information for {project_name}.\n", "black", "white")
    else:
        add_message("Chatbot: I'm sorry, I don't understand your query.\n", "black", "white")

# Function to add messages to the chat
def add_message(message, background, text_color):
    chat_output_text.config(state="normal", bg=background, fg=text_color)  # Allow editing
    chat_output_text.insert(tk.END, message)
    chat_output_text.config(state="disabled")  # Disable editing
    chat_output_text.see(tk.END)  # Scroll to the latest message

# Create the main window
root = tk.Tk()
root.title("Prakalp Nirdesh")

root = tk.Tk()
root.title("Prakalp Nirdesh")
root.geometry("800x600")

# Customize the title
title_label = Label(root, text="Prakalp Nirdesh", font=("product sans", 20, "italic"), fg="green")
title_label.pack()

# Create and place UI elements
# Create and place UI elements
user_input_label = Label(root, text="Enter your query:",font=("product sans", 15, "italic"),fg = "black")
user_input_label.pack()

custom_font = ("product sans", 15)  # You can adjust the font size as needed

user_input_entry = Entry(root, width=100, font=custom_font)
user_input_entry.pack()

# Set the background color of the Entry widget to light pink
user_input_entry.configure(bg='light grey')

chat_button = Button(root, text="Submit",font=("product sans", 15, "italic"),fg = "black", command=handle_button_click)
chat_button.pack()

chat_output_text = tk.Text(root, wrap="word", width=100, height=50, state="disabled")
chat_output_text.pack()


root.mainloop()

# Close the database connection
db.close()


abc_loading
Query: SELECT Paths FROM prakalp_nirdesh WHERE Project_Name like %s


In [6]:
import tkinter as tk
from tkinter import Entry, Button, Label
import mysql.connector
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# ... (Previous code remains the same)
db = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password1,
    database="Krishna"
)

cursor = db.cursor()

# Preprocess user query
def preprocess_query(user_query):
    words = nltk.word_tokenize(user_query)
    words = [word for word in words if word not in stopwords.words('english')]
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return words

def get_project_info(project_name, info_type):
    try:
        if info_type not in ["path", "procedure", "deadline", "head",'details','start','end','Date','member','name','mail','notes']:
            return "Invalid info type. Please specify 'path', 'procedure', 'deadline', or 'head'."
        
        if 'path' in info_type:
            column_name = 'Paths'
        elif 'procedure' in info_type:
            column_name = 'ProjectProcedure'
        elif 'member' in info_type:
            column_name = 'Project_Member'
        elif 'details' in info_type:
            column_name = '*'
        elif 'Date' in info_type or 'deadline' in info_type:
            column_name = 'End_Date'
        elif 'mail' in info_type:
            column_name = 'Email'
        elif 'name' in info_type:
            column_name = 'Project_Name'
        elif 'notes' in info_type:
            column_name = 'Notes'
    
        print(project_name)
        if 'details' in info_type:
            query = f"SELECT {column_name} FROM prakalp_nirdesh WHERE Project_Name like %s"
            print("Query:", query)
            cursor.execute(query, ("%" + project_name + "%",))
            info = cursor.fetchall()
            if info:
                return info[0]
            else:
                return f"No {info_type} found for {project_name}."
        else:
            query = f"SELECT {column_name} FROM prakalp_nirdesh WHERE Project_Name like %s"
            print("Query:", query)
            cursor.execute(query, ("%" + project_name + "%",))

            info = cursor.fetchall()
            if info:
                return info[0]
            else:
                return f"No {info_type} found for {project_name}."
        print(query)
    except Exception as e:
        print("An error occurred while querying the database:", str(e))
        return "An error occurred while fetching information."

# Function to handle button click
def handle_button_click():
    user_input = user_input_entry.get()
    user_input_entry.delete(0, tk.END)
    user_input = user_input.lower()

    if 'path' in user_input:
        column_name = 'path'
    elif 'procedure' in user_input:
        column_name = 'procedure'
    elif 'head' in user_input or 'member' in user_input or 'person' in user_input:
        column_name = 'member'
    elif 'details' in user_input:
        column_name = 'details'
    elif 'start' in user_input or 'end' in user_input or 'date' in user_input or 'deadline' in user_input:
        column_name = 'deadline'
    elif 'mail' in user_input or 'gmail' in user_input or 'email' in user_input:
        column_name = 'mail'
    elif 'name' in user_input:
        column_name = 'name'
    elif 'notes' in user_input:
        column_name = 'notes'
        
    # Check if the user input contains the query format "of Project1" or "of Project 1"
    if "project" in user_input:
        project_name = user_input.split("project")[1].strip()
        info = get_project_info(project_name, column_name)  # Assuming the user wants the path
        if info:
            query_project_name = f"select Project_Name from krishna.prakalp_nirdesh where project_name like %s;"
            cursor.execute(query_project_name, ("%" + project_name + "%",))
            project_Name1 = cursor.fetchone()
            add_message(f"{user_input}", sender="User")
            add_message(f"The {column_name} for {project_Name1} is:", sender="Chatbot")
            if 'details' not in column_name:
                add_multiline_message(info, sender="Chatbot")
            if column_name == 'details':
                add_message(f"Chatbot: The details for {project_Name1} are: \n", sender="Chatbot")
                add_message(details_split(),sender = 'chatbot')
        else:
            add_message(f"Chatbot: I couldn't find information for {project_name}.", sender="Chatbot")
    else:
        add_message("Chatbot: I'm sorry, I don't understand your query.", sender="Chatbot")
        

# Function to add multiline messages
def add_multiline_message(text, sender="Chatbot"):
    text_lines = text[0].split('\n')
    for line in text_lines:
        if line:
            add_message(line, sender=sender)
    
def add_message(message,sender,text_color = 'red',font=("Product Sans", 15,'bold')):
    chat_output_text.config(state="normal",bg = 'black' ,fg = 'white',font=font)  # Allow editing, no background
    if sender == "Chatbot":
        chat_output_text.tag_configure("chatbot", foreground=text_color,background = 'green')
        chat_output_text.insert(tk.END, f"{sender}:", "chatbot")
    elif sender == "User":
        chat_output_text.tag_configure("user", foreground=text_color,background = 'yellow')
        chat_output_text.insert(tk.END, f"{sender}:", "user")
    chat_output_text.insert(tk.END, f" {message}\n")
    chat_output_text.config(state="disabled", font=font)  # Disable editing
    chat_output_text.see(tk.END)

# Function to get column names from a table
def get_column_names(table_name):
    query = f"SHOW COLUMNS FROM {table_name}"
    cursor.execute(query)
    columns = [column[0] for column in cursor.fetchall()]
    return columns

# Function to execute a query and fetch results
def execute_query(query):
    cursor.execute(query)
    results = cursor.fetchall()
    return results
def details_split():
    for i in range(0,8):
        add_message(f"{columns[i]}: {results[0][i]}\n", sender='chatbot')
# Zip column names and results
columns = get_column_names('prakalp_nirdesh')
results = execute_query(f"SELECT * FROM prakalp_nirdesh")
zipped_data = list(zip(columns, results))
# Function to add messages to the chat with visual differentiation
# Create Main Window
root = tk.Tk()
root.title("Prakalp Nirdesh")

root.geometry("800x600")

# Customize the title
title_label = Label(root, text="Prakalp Nirdesh", font=("product sans", 20, "italic"), fg="green")
title_label.grid(row=0,column = 0)

# Create and place UI elements
#user_input_label = Label(root, text="Enter your query:", font=("product sans", 15, "italic"), fg="black")
#user_input_label.grid(row= 5,column = 0)

custom_font = ("product sans", 15)  # You can adjust the font size as needed

user_input_entry = Entry(root, width=100, font=custom_font)
user_input_entry.grid(row = 5,column=0)

# Set the background color of the Entry widget to light pink
user_input_entry.configure(bg='light grey')

chat_button = Button(root, text=">>", font=("product sans", 15, "italic"), fg="black", command=handle_button_click)
chat_button.grid(row =5,column = 1)

chat_output_text = tk.Text(root, wrap="word", width=100, height=28, state="disabled")
chat_output_text.grid(row=1, column=0)

root.mainloop()

# Close the database connection
db.close()

validate
Query: SELECT * FROM prakalp_nirdesh WHERE Project_Name like %s
